<a href="https://colab.research.google.com/github/NewCodeLearner/HandsOnLLM-Projects/blob/main/09_First_LLM_agent_using_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Chapter, we will learn how to code your first LLM Agent

In particular, we learn about the following:

(1) What are agents and how do they differ from just LLMs?  
(2) The role of tools in deploying agents  
(3) The “ReAct” framework (reasoning-acting)  
(4) The 3 pillars of how agents operate: thoughts - actions - observations (tao)  
(5) Coding your first LLM agent to solve a practical problem  

Links:
Agents in Langchain : https://python.langchain.com/docs/how_to/#agents
Langchain Agents Tools: https://github.com/kyrolabs/awesome-langchain?utm_source=chatgpt.com


In [1]:
# Installing Ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
# needed for tunneling via ngrok
%env OLLAMA_HOST=0.0.0.0

env: OLLAMA_HOST=0.0.0.0


In [3]:
# Running Ollama and serving on localhost:11434
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [4]:
# Downloading a model from https://ollama.com/search
!ollama pull llama3.1:8b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏ 6.5 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   1% ▕▏  41 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   1% ▕▏  61 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   2% ▕▏ 118 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   3% ▕▏ 142 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   4% ▕▏ 183 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   5% ▕▏ 223 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   5% ▕▏ 239 MB/4.9 GB  239 MB/s   

In [5]:
# Check the available models
!ollama list

NAME           ID              SIZE      MODIFIED           
llama3.1:8b    46e0c10c039e    4.9 GB    About a minute ago    


In [6]:
import psutil

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [ ]:
import urllib.request

def query_model(
    prompt,
    model="llama3.1:8b",
    url="http://localhost:11434/api/chat"
):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }


    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

In [10]:
!ollama run llama3.1:8b "what do Llamas eat?"

⠙ Llamas are herbivores, which means they primarily feed on plant-based foods. Their diet consists of:

1. **Grasses**: They love to graze on various types of grasses, including bunchgrasses, blue grama, and switchgrass.
2. **Leaves**: Llamas enjoy munching on leaves from trees and shrubs, such as willow, cottonwood, and juniper.
3. **Hay**: High-quality hay, like timothy or alfalfa, is a staple in a llama's diet. It provides fiber, energy, and essential nutrients.
4. **Fruits**: Apples, carrots, and other fruits are given to llamas as treats or added to their meals for variety and nutrition.
5. **Grains**: Oats, barley, and corn can be included in small amounts (about 10% of their diet) to provide additional energy and nutrients.

In the wild, llamas will also eat:

* Mushrooms
* Bark
* Twigs
* Flowers

Domesticated llamas typically receive a balanced diet that includes a mix of hay, grains, and supplements. It's essential for llama owners to consult with a veterinarian or experienced